In [34]:
%cd /content
!pwd
!ls


/content
/content
mask_project.zip  sample_data


In [35]:
!ls *.zip


mask_project.zip


In [36]:
!unzip -q "mask_project.zip" -d mp
!ls mp


mask_project


In [37]:
%cd /content
!ls *.zip


/content
mask_project.zip


In [38]:
import os

base = "/content/mp"
dataset_path = None

for root, dirs, files in os.walk(base):
    if "with_mask" in dirs and "without_mask" in dirs:
        dataset_path = root
        break

print("✅ dataset_path =", dataset_path)


✅ dataset_path = /content/mp/mask_project/Dataset-example


In [40]:
import os, cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

IMG_SIZE = 128
EPOCHS = 10
BATCH_SIZE = 16

dataset_path = "/content/mp/mask_project/Dataset-example"
with_mask_dir = os.path.join(dataset_path, "with_mask")
without_mask_dir = os.path.join(dataset_path, "without_mask")

data, labels = [], []

def load(folder, label):
    for f in os.listdir(folder):
        p = os.path.join(folder, f)
        img = cv2.imread(p)
        if img is None:
            continue
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        data.append(img)
        labels.append(label)

load(with_mask_dir, 1)
load(without_mask_dir, 0)

data = np.array(data, dtype="float32") / 255.0
labels = to_categorical(labels, 2)

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42, shuffle=True, stratify=labels.argmax(axis=1)
)

# ✅ MobileNetV2 base model
base = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet")
base.trainable = False  # freeze

x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
out = Dense(2, activation="softmax")(x)

model = Model(inputs=base.input, outputs=out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, epochs=EPOCHS, validation_data=(X_test, y_test), batch_size=BATCH_SIZE)

loss, acc = model.evaluate(X_test, y_test)
print("✅ Final Accuracy:", acc)

model.save("mask_model_mobilenet.h5")
print("✅ Saved: mask_model_mobilenet.h5")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.7273 - loss: 0.6630 - val_accuracy: 1.0000 - val_loss: 0.0627
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step - accuracy: 0.8182 - loss: 0.4414 - val_accuracy: 1.0000 - val_loss: 0.0607
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.7273 - loss: 0.4398 - val_accuracy: 1.0000 - val_loss: 0.0644
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - accuracy: 0.7273 - loss: 0.4362 - val_accuracy: 1.0000 - val_loss: 0.0594
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - accuracy: 0.9091 - loss: 0.2630 - val_accuracy: 1.0000 - val_loss: 0.0550
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - accuracy: 0.9091 - loss: 0.2335 - val_accuracy: 1.0000 - val_loss: 0.0509
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 1.0000 - loss: 0.0819 - val_accuracy: 1.0000 - val_loss: 0.0480
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 1.0000 - l

✅ Final Accuracy: 1.0
✅ Saved: mask_model_mobilenet.h5
